In [1]:
#!pip install textract
#!pip install colorama

In [ ]:
import os
import re
import json
import textract
import requests
import colorama
from bs4 import BeautifulSoup
from operator import itemgetter
from urllib.parse import urljoin,urlparse

In [ ]:
def DataFromFile(path):
  text = textract.process(path)
  text = text.decode("utf-8") 
  text=str(text).replace('\n',' ')
  return text

In [ ]:
def SinglePagePreprocessedContent(url):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, features="html.parser")
    dictionary = {}

    for tag in soup.find_all(True):
        if tag.name in dictionary:
            dictionary[tag.name] +=1
        else:
            dictionary[tag.name] = 1

    keys=list(dictionary.keys())
    newDict={}
    if 'html' in dictionary.keys():
          keys.remove('html')
    if 'head' in dictionary.keys():
          keys.remove('head')

    if 'body' in dictionary.keys():
          keys.remove('body')

    if 'script' in dictionary.keys():
          keys.remove('script')

    if 'meta' in dictionary.keys():
          keys.remove('meta')

    if 'link' in dictionary.keys():
          keys.remove('link')

    if 'style' in dictionary.keys():
          keys.remove('style')    
    
    if 'mp4' in dictionary.keys():
          keys.remove('mp4')    
    
    if 'img' in dictionary.keys():
          keys.remove('img')    


    for tag in keys:
        data=soup.find_all(tag)
        newDict[tag]=list()        
        for textData in data:
            value=textData.text
            value=re.sub("\s\s+" , " ", value)
            value=value.strip()
            value=re.sub("\n" , " ", value)
            if '((function()' not in value:
              newDict[tag].append(value)
        newDict[tag]=set(newDict[tag])        
        newDict[tag]=list(filter(lambda a: a != '', newDict[tag]))
        
            
    return newDict

In [ ]:
def DefineDictionaryHTML():
  Dictionary={
      'Page Link': '',
      'Title':'',
      'Content':{},
  }
  return Dictionary

In [ ]:
def DictionaryFillerHTML(url):
  Dictionary=DefineDictionaryHTML()
  Dictionary['Page Link']=url
  HTMLDict=SinglePagePreprocessedContent(url)
  if 'title' in HTMLDict.keys():
    Dictionary['Title']=HTMLDict['title']
  Dictionary['Content']=HTMLDict

  return Dictionary


In [ ]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_website_links(url):

    urls = set()
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            continue
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            continue
        if href in internal_urls:
            continue
        if domain_name not in href:
            if href not in external_urls:
                print(f"{GRAY}[!] External link: {href}{RESET}")
                external_urls.add(href)
            continue
        print(f"{GREEN}[*] Internal link: {href}{RESET}")
        urls.add(href)
        internal_urls.add(href)
    return urls


def crawl(url):
    global total_urls_visited
    total_urls_visited += 1
    print(f"{YELLOW}[*] Crawling: {url}{RESET}")
    links = get_all_website_links(url)
    for link in links:
        crawl(link)
url="https://www.anna-dsb.com/"
colorama.init()
GREEN = colorama.Fore.GREEN
GRAY = colorama.Fore.LIGHTBLACK_EX
RESET = colorama.Fore.RESET
YELLOW = colorama.Fore.YELLOW
total_urls_visited = 0
internal_urls = set()
external_urls = set()
crawl(url)


In [ ]:
links=internal_urls
delLinks=links
FinalData=[]
for link in links:
  if '.mp4' in link:
    delLinks.remove(link)

links=delLinks


In [ ]:
for link in links:
  FinalData.append(DictionaryFillerHTML(link))

In [ ]:
Folder_Name="WebScrapper"
os.mkdir(Folder_Name)
path=os.path.join(os.getcwd(),Folder_Name)
DataInOneFile="DataInOneFile"
DataInSeperatedFile="DataInSeperatedFile"
pathDataInOneFile=os.path.join(path,DataInOneFile)
pathDataInSeperatedFile=os.path.join(path,DataInSeperatedFile)
os.mkdir(pathDataInOneFile)
os.mkdir(pathDataInSeperatedFile)

In [ ]:
#DataInOneFile
File_Name="data.json"
npath=os.path.join(pathDataInOneFile,File_Name)
with open(npath, "w") as outfile:
    json.dump(FinalData, outfile)


In [ ]:
#DataInSeperatedFile

count=1
for link in links:
    File_Name=str(count)+".json"
    mpath=os.path.join(pathDataInSeperatedFile,File_Name)
    with open(mpath, "w") as outfile:
        json.dump(FinalData[count-1], outfile)
    count=count+1
